In [ ]:
# Convert the raw data to occupancy grid

In [1]:
import os
import ctypes
import numpy as np
from tqdm import tqdm
from pathlib import Path
import ipdb
from pcdet.models import build_network, load_data_to_gpu
from pcdet.datasets import build_dataloader
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.utils import common_utils

/home/pengcheng/anaconda3/envs/openpcdet/lib/python3.8/site-packages/tqdm-4.66.1-py3.8.egg/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Read the config file
config = 'cfgs/waymo_models/centerpoint_dyn_pillar020_1x.yaml'
cfg_from_yaml_file(config, cfg)
cfg.TAG = Path(config).stem
cfg.EXP_GROUP_PATH = '/'.join(config.split('/')[1:-1])  # remove 'cfgs' and 'xxxx.yaml'
np.random.seed(1024)
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
dist_test = False
total_gpus = 1
batch_size = 1
workers = 4

# Create results directory
output_dir = cfg.ROOT_DIR / 'output/occupancy_grid/preprocessed_50cm/test/data'
output_dir.mkdir(parents=True, exist_ok=True)

# Build the dataloader
logger = common_utils.create_logger()
test_set, test_loader, sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=batch_size,
    dist=dist_test, workers=workers, logger=logger, training=False)
class_names = test_set.class_names
print(f'Total number of samples: \t{len(test_set)}')

# Convert the raw data to *bin format
progress_bar = tqdm(total=len(test_loader), leave=True, desc='eval', dynamic_ncols=True)
for idx, data_dict in enumerate(test_loader):
    # print(data_dict['points'])
    pc = data_dict['points'][:, 1:]
    # for i in tqdm(range(len(pc))):
    #     if pc[i][0] == 0:
    #         continue
    #     else:
    #         break
    
    # ipdb.set_trace()
    # print(pc.shape)    
    file_name = f"{idx:05d}.bin"
    file_path = f"../output/occupancy_grid/preprocessed_50cm/test/data/{file_name}" 
    pc.tofile(file_path)
    progress_bar.update()


2023-11-28 17:27:04,476   INFO  Loading Waymo dataset


2023-11-28 17:27:07,213   INFO  Total skipped info 0
2023-11-28 17:27:07,216   INFO  Total samples for Waymo dataset: 39987


Total number of samples: 	39987


eval: 100%|██████████| 39987/39987 [40:04<00:00, 26.45it/s]  

eval: 100%|██████████| 39987/39987 [40:20<00:00, 26.45it/s]

In [13]:
np.load('../data/waymo/waymo_processed_data_v0_5_0/segment-10017090168044687777_6380_000_6400_000_with_camera_labels/0000.npy')

array([[-1.9172607e+01, -6.5429688e-02,  2.9200058e+00,  1.9042969e-01,
         1.4931070e+00, -1.0000000e+00],
       [-1.7223999e+01, -1.5625000e-02,  2.8512726e+00,  3.5888672e-02,
         7.6119177e-02, -1.0000000e+00],
       [-1.8926514e+01,  3.2226562e-02,  2.9111710e+00,  1.6308594e-01,
         9.3099612e-01, -1.0000000e+00],
       ...,
       [-1.4748725e+00, -1.2011310e+00,  5.8255494e-03,  2.1362305e-02,
         6.3823003e-01, -1.0000000e+00],
       [-1.3140347e+00, -1.1765740e+00,  1.5360743e-02,  2.0874023e-02,
         9.6027273e-01, -1.0000000e+00],
       [-1.1180688e+00, -1.1016902e+00,  2.4845809e-02,  1.4404297e-02,
         0.0000000e+00, -1.0000000e+00]], dtype=float32)

In [3]:
import ctypes
import glob
import os
import numpy as np
from tqdm import tqdm
# Point cloud data N*5
# 5 = 3 + 1 + 1 + 1 = points_all, points_intensity, points_elongation


# Convert *bin format data to occupancy grid
rows = 300
cols = 300
height = 62
data_dir = '../output/occupancy_grid/preprocessed_50cm/test/data'
output_dir = '../output/occupancy_grid/preprocessed_50cm/test'
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, 'occ'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'intensity'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'elongation'), exist_ok=True)

SharedLib = ctypes.cdll.LoadLibrary('./LidarPreprocess.so')
frames = sorted(glob.glob(os.path.join(data_dir, '*.bin')))

for frame in tqdm(frames):
    # Point cloud data N*6
    # 5 = 3 + 1 + 1 + 1 = points_all, points_intensity, points_elongation
    npdata = np.zeros((rows, cols, height), dtype=np.float32)
    cdata = ctypes.c_void_p(npdata.ctypes.data)
    apath = bytes(frame, 'utf-8')
    SharedLib.createTopViewMaps(cdata, apath)
    occ = npdata[..., :height - 2].astype(np.bool)

    occ_bits = np.packbits(occ)
    intensity = npdata[..., -2].astype(np.float32)
    elongation = npdata[..., -1].astype(np.float32)

    name = os.path.basename(frame)[:-4]

    occ_name = os.path.join(output_dir, 'occ', name + '.npy')
    intensity_name = os.path.join(output_dir, 'intensity', name + '.npy')
    elongation_name = os.path.join(output_dir, 'elongation', name + '.npy')
    
    np.save(occ_name, occ_bits)
    np.save(intensity_name, intensity)
    np.save(elongation_name, elongation)

100%|██████████| 39987/39987 [35:51<00:00, 18.58it/s]
